In [1]:
# append code path
import sys
sys.path.append('../../../code')

# common modules
import numpy as np
from time import time
from swimnetworks import Linear, Dense
from activations import Activations 

# swim solving Hamiltonian systems
from swim import solve_swim_hamiltonian, weight_sampling
from NN.extreme_learning_machine import ELM

# physical systems
from galactic_model import GalacticModel

# utils for visualization
from draw import draw_2d

## **6-D Example**

The following function is a Hamiltonian with 6 degrees of freedom. Therefore, let's define our general Hamiltonian system!

A Hamiltonian system is determined through a function called ***Hamiltonian***. 

### $$H: E \rightarrow \mathbb{R} $$  

The Hamiltonian incorporates the following set of equations that dictates how a dynamical system changes and evolves over time.  

### $$ \frac{\partial q(t)}{\partial t} = \frac{\partial H(q(t),p(t))}{\partial p} $$

### $$ \frac{\partial p(t)}{\partial t} = -\frac{\partial H(q(t),p(t))}{\partial q} $$

Where $(q,p) \in \mathbb{R}^{2n}$, and $n$ is the number of degrees of freedom of the system. The equations above can be restated as the following to construct a PDE for $H$ at every $(q,p) \in E $,

### $$\begin{bmatrix} 0 & I \\ -I & 0 \end{bmatrix} \cdot \nabla H(q,p) - v(q,p) $$
### $$ = \begin{bmatrix} 0 & I \\ -I & 0 \end{bmatrix} \cdot \begin{bmatrix} \frac{\partial H(q,p)}{\partial q} \\ \frac{\partial H(q,p)}{\partial p} \end{bmatrix} - \begin{bmatrix} \frac{\partial q}{\partial t} \\ \frac{\partial p}{\partial t} \end{bmatrix} $$
### $$ = \begin{bmatrix} \frac{\partial H(q,p)}{\partial p} \\ -\frac{\partial H(q,p)}{\partial q} \end{bmatrix} - \begin{bmatrix} \frac{\partial q}{\partial t} \\ \frac{\partial p}{\partial t} \end{bmatrix} = 0 $$

where $I \in \mathbb{R}^{n \times n}$ is the identity matrix and $v$ is the vector field on $E$. 

### **Galactic Model**

The particular function that we want to approximate is the Galactic model. This system has 3 degrees of freedom, i.e. $n = 3$, so $(q,p) \in \mathbb{R}^6$ where $q = (q_1, q_2, q_3)$ and $p = (p_1, p_2, p_3)$, and for the PDE of the Hamiltonian please refer to the code.

The Hamiltonian for this particular system is the following:

In [2]:
galactic_model = GalacticModel()
H = galactic_model.H
dH = galactic_model.dH

# We will plot the phase space of our example system 
# to get an overview of the global view of the phase 
# space that we would like to approximate.

# num. of points in the grid, grid dimension is (N_q1 x N_q2 x N_p1 x N_p2)
N_q1, N_q2, N_q3, N_p1, N_p2, N_p3 = 10, 10, 10, 10, 10, 10 
N = N_q1 * N_q2 * N_q3 * N_p1 * N_p2 * N_q3 
q1_lim, q2_lim, q3_lim = [0, 10], [0, 10], [0, 10]
p1_lim, p2_lim, p3_lim = [0, 10], [0, 10], [0, 10]
 
q1_range, q2_range, q3_range = np.linspace(q1_lim[0], q1_lim[1], N_q1), np.linspace(q2_lim[0], q2_lim[1], N_q2), np.linspace(q3_lim[0], q3_lim[1], N_q3)
p1_range, p2_range, p3_range = np.linspace(p1_lim[0], p1_lim[1], N_p1), np.linspace(p2_lim[0], p2_lim[1], N_p2), np.linspace(p3_lim[0], p3_lim[1], N_p3)

q1_grid, q2_grid, q3_grid, p1_grid, p2_grid, p3_grid = np.meshgrid(q1_range, q2_range, q3_range, p1_range, p2_range, p3_range)

# prepare the input x = [ x_1, .., x_{N} ] of shape (N,6) 
x = np.column_stack([q1_grid.flatten(), q2_grid.flatten(), q3_grid.flatten(), p1_grid.flatten(), p2_grid.flatten(), p3_grid.flatten()])
y = H(x) # of shape (N,)
dx = dH(x) # of shape (N, 6)
assert x.shape == (N, 6)
assert y.shape == (N,)
assert dx.shape == (N, 6)

# in order to plot the values, we need to have output of H(x) in
# shape (N_q3, N_q2, N_q1, N_p1, N_p2, N_p3), which means we get the value H(x)[i,j,k,l,m,n] for the 
# grid point at (q2_grid[k,j,i,l,m,n], q1_grid[k,j,i,l,m,n], p1_grid[k,j,i,l,m,n], p2_grid[k,j,i,l,m,n])
print(y.shape)
y_grid = y.reshape((N_q3, N_q2, N_q1, N_p1, N_p2, N_p3))
dx_grid = dx.reshape((N_q3, N_q2, N_q1, N_p1, N_p2, N_p3, 6))

# visualize H(x)
# fix q1 = 0, p1 = 0 by choosing j = N_q1 / 2 and k = N_p1 / 2
#print("y_grid[:,..,..,:] shape:", y_grid[:, N_q1 // 2, N_p1 // 2, :].shape)
#print("N_q1 / 2 == " + str(N_q1 // 2))
#print("y_grid[0, N_q1//2]")

# 5 poincare maps
#plot_data_1 = y_grid[:, N_q1//2, N_p1//2, :].reshape((N_p2, N_q2)) 
#draw_2d(plot_data_1, q2_range, p2_range, extent=[q2_lim[0],q2_lim[1],p2_lim[0],p2_lim[1]], showcontour=True, xlabel='q2', ylabel='p2', aspect=2.5, title="H(x) with q1,p1 near 0")

#plot_data_2 = y_grid[N_q2//2, :, :, N_p2//2].reshape((N_p1, N_q1)) 
#draw_2d(plot_data_2, q1_range, p1_range, extent=[q1_lim[0],q1_lim[1],p1_lim[0],p1_lim[1]], showcontour=True, xlabel='q1', ylabel='p1', aspect=2.5, title="H(x) with q2,p2 near 0")

#plot_data_3 = y_grid[:, :, N_p1//2, N_p2//2].reshape((N_q1, N_q2)) 
#draw_2d(plot_data_3, q2_range, q1_range, extent=[q2_lim[0],q2_lim[1],q1_lim[0],q1_lim[1]], showcontour=True, xlabel='q2', ylabel='q1', aspect=1, title="H(x) with p1,p2 near 0")

#plot_data_4 = y_grid[N_q2//2, N_q1//2, :, :].reshape((N_p2, N_p1)) 
#draw_2d(plot_data_4, p1_range, p2_range, extent=[p1_lim[0],p1_lim[1],p2_lim[0],p2_lim[1]], showcontour=True, xlabel='p1', ylabel='p2', aspect=1, title="H(x) with q1,q2 near 0")

#plot_data_5 = y_grid[40, 0, :, :].reshape((N_p2, N_p1)) 
#draw_2d(plot_data_5, p1_range, p2_range, extent=[p1_lim[0],p1_lim[1],p2_lim[0],p2_lim[1]], showcontour=True, xlabel='p1', ylabel='p2', aspect=1, title="H(x) with q1,q2=" + str(q2_grid[40,0,0,0]) + "," + str(q1_grid[0,0,0,0]))

(1000000,)


/home/gergedan/WS24/thesis/code/notebooks/galactic_model/../../../code/galactic_model.py:18: RuntimeWarning: invalid value encountered in log
  return (self.v0 / 2) * np.log(q1**2 + self.alpha * q2**2 + self.beta * q3**2 - self.l * q1**3 + self.cb**2 + 1e-100)


In [3]:
np.random.seed(123495)

# Sample training dataset
# hint: use sampled 1000 random points in [-pi/2,pi/2]x[-1/2,1/2]
noise = 1e-5
#K_q1, K_q2, K_p1, K_p2 = 25, 25, 5, 5 # 15625
K_q1, K_q2, K_q3, K_p1, K_p2, K_p3 = 5, 5, 5, 5, 5, 5 # ??
K = K_q1 * K_q2 * K_q3 * K_p1 * K_p2 * K_p3
q1_lim_train, q2_lim_train, q3_lim_train = [0, 10], [0, 10], [0, 10]
p1_lim_train, p2_lim_train, p3_lim_train = [0, 10], [0, 10], [0, 10]
q1_range_train, q2_range_train, q3_range_train = np.linspace(q1_lim_train[0], q1_lim_train[1], K_q1)+np.random.randn(K_q1)*noise*1e-3, np.linspace(q2_lim_train[0], q2_lim_train[1], K_q2)+np.random.randn(K_q1)*noise*1e-3, np.linspace(q3_lim_train[0], q3_lim_train[1], K_q3)+np.random.randn(K_q3)*noise*1e-3
p1_range_train, p2_range_train, p3_range_train = np.linspace(p1_lim_train[0], p1_lim_train[1], K_p1)+np.random.randn(K_p1)*noise*1e-3, np.linspace(p2_lim_train[0], p2_lim_train[1], K_p2)+np.random.randn(K_p2)*noise*1e-3, np.linspace(p3_lim_train[0], p3_lim_train[1], K_p3)+np.random.randn(K_p3)*noise*1e-3
q1_grid_train, q2_grid_train, q3_grid_train, p1_grid_train, p2_grid_train, p3_grid_train = np.meshgrid(q1_range_train, q2_range_train, q3_range_train, p1_range_train, p2_range_train, p3_range_train) 
x_train_values = np.column_stack([q1_grid_train.flatten(), q2_grid_train.flatten(), q3_grid_train.flatten(), p1_grid_train.flatten(), p2_grid_train.flatten(), p3_grid_train.flatten()])
x_train_derivs = dH(x_train_values) # of shape (K,6)
x0 = np.array([0,0,0,0,0,0])
f0 = H(x0.reshape(-1,6))

# Sample points that we want to predict -- we use the same ones in the above cell which is just x

In [4]:
# ELM
random_seed = 1
# define the parameters
M = 1000
regularization_scale = 1e-8 # for the lstsq, same as bertalan-2019
random_seed = 1
sample_uniformly = True
parameter_sampler = 'random' # 'relu', 'tanh', 'random'
activation = 'relu'

swim_model = solve_swim_hamiltonian(
    x_train_values, x_train_derivs, x0, f0,
    n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, sample_uniformly=sample_uniformly, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
    include_bias=True
)

# swim model approximation is
phi_2 = swim_model.transform(x).reshape(-1) # (N)
train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
eval_mse_loss = np.sum((y - phi_2)**2) / N
print("train loss:", str(train_mse_loss))
print("eval loss:", str(eval_mse_loss))

train loss: nan
eval loss: nan


In [5]:
# SWIM with random data picking
parameter_sampler = 'relu' # 'relu', 'tanh', 'random'

swim_model = solve_swim_hamiltonian(
    x_train_values, x_train_derivs, x0, f0,
    n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, sample_uniformly=sample_uniformly, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
    include_bias=True
)

# swim model approximation is
phi_2 = swim_model.transform(x).reshape(-1) # (N)
train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
eval_mse_loss = np.sum((y - phi_2)**2) / N
print("train loss:", str(train_mse_loss))
print("eval loss:", str(eval_mse_loss))

train loss: nan
eval loss: nan


In [6]:
# SWIM with random data picking
swim_model = weight_sampling(
    x_train_values, x_train_derivs, x0, f0,
    n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
    include_bias=True
)

# swim model approximation is
phi_2 = swim_model.transform(x).reshape(-1) # (N)
train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
eval_mse_loss = np.sum((y - phi_2)**2) / N
print("train loss:", str(train_mse_loss))
print("eval loss:", str(eval_mse_loss))

train loss: nan
eval loss: nan


In [7]:
# ELM
parameter_sampler = 'random' # 'relu', 'tanh', 'random'
activation = 'tanh'

swim_model = solve_swim_hamiltonian(
    x_train_values, x_train_derivs, x0, f0,
    n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, sample_uniformly=sample_uniformly, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
    include_bias=True
)

# swim model approximation is
phi_2 = swim_model.transform(x).reshape(-1) # (N)
train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
eval_mse_loss = np.sum((y - phi_2)**2) / N
print("train loss:", str(train_mse_loss))
print("eval loss:", str(eval_mse_loss))

train loss: nan
eval loss: nan


In [ ]:
# SWIM with random data picking
parameter_sampler = 'tanh' # 'relu', 'tanh', 'random'

swim_model = solve_swim_hamiltonian(
    x_train_values, x_train_derivs, x0, f0,
    n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, sample_uniformly=sample_uniformly, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
    include_bias=True
)

# swim model approximation is
phi_2 = swim_model.transform(x).reshape(-1) # (N)
train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
eval_mse_loss = np.sum((y - phi_2)**2) / N
print("train loss:", str(train_mse_loss))
print("eval loss:", str(eval_mse_loss))

In [ ]:
# SWIM with random data picking
swim_model = weight_sampling(
    x_train_values, x_train_derivs, x0, f0,
    n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
    include_bias=True
)

# swim model approximation is
phi_2 = swim_model.transform(x).reshape(-1) # (N)
train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
eval_mse_loss = np.sum((y - phi_2)**2) / N
print("train loss:", str(train_mse_loss))
print("eval loss:", str(eval_mse_loss))